In [1]:
import sys
sys.path.append('../../')

import jax
import jax.numpy as np
import numpy as onp
import os
import matplotlib.pyplot as plt
from discretization2D import *
from solver import *
from plot import *

os.environ["CUDA_VISIBLE_DEVICES"]="1"
jax.config.update("jax_enable_x64", True)
%matplotlib notebook

In [2]:
### 2D box mesh
domain_x = 1.
domain_y = 1.
Nx = 128
Ny = 128
cell_num = Nx*Ny
num_nodes = (Nx + 1) * (Ny + 1)
x = onp.linspace(0., domain_x, Nx + 1)
y = onp.linspace(0., domain_y, Ny + 1)
xx, yy = onp.meshgrid(x, y, indexing='ij')
points = onp.vstack([xx.reshape(-1), yy.reshape(-1)]).T

xc = np.array((xx[1:, 1:] + xx[:-1, 1:])/2)[:, :,None]
yc = np.array((yy[1:, 1:] + yy[1:, :-1])/2)[:, :,None]

dX = np.array([domain_x/Nx, domain_y/Ny])

### BC: -x,+x,-y,+y; 0 for dirichlet, 1 for neumann
BCs = [0,0,0,0]
v1 = np.array([0.,0.,])
v2 = np.array([1.,0.,])
values = [v1,v1,v1,v2]
BC_info = [BCs,values]

In [3]:
### example 1: transient solver, implicit 
def compute_residual(vel,p,vel0,vel0_f):
    p_b = get_bound_values(p,[[1,1,1,1],[0,0,0,0]],dX)
    vel_b = get_bound_values(vel,BC_info,dX)
    
    res1 = ((vel-vel0)/dt*dX[0]*dX[1] + 
            convection(vel,vel_b,vel0_f,dX) - 
            diffusion(vel,vel_b,visco,dX) + 
            gradient(p,p_b,dX)*dX[0]*dX[1])
    
    u_f,v_f = get_face_values(vel,BC_info,dX)
    res2 = ((u_f[1:,:]-u_f[:-1,:])*dX[1] + (v_f[:,1:]-v_f[:,:-1])*dX[0])  
    
    return np.concatenate((res1.flatten(),res2.flatten()))

@jax.jit
def time_integration(vel0,vel0_f):
    def A_fn(f):
        vel = f[:Nx*Ny*2].reshape((Nx,Ny,-1))
        p = f[Nx*Ny*2:].reshape((Nx,Ny,-1))
        return compute_residual(vel,p,vel0,vel0_f)
    
    
    dofs = np.zeros(Nx*Ny*3)
    dofs,res = solver_linear(A_fn,dofs)
    
    vel0 = dofs[:Nx*Ny*2].reshape(Nx,Ny,-1)
    p = dofs[Nx*Ny*2:].reshape(Nx,Ny,-1)
    vel0_f = get_face_values(vel0,BC_info,dX)
    return vel0,vel0_f,p,res

In [ ]:
%%time
#Re = 100
visco = 0.001
dt = 0.005
vel0 = np.zeros((Nx,Ny,2))
p0 = np.zeros((Nx,Ny,1))
vel0_f = get_face_values(vel0,BC_info,dX)
for i in range(0,3000):
    vel0,vel0_f,p0,res = time_integration(vel0,vel0_f)
    
    if res > 1e-5:
        print(f"res_norm = {res}, check the convergence!")
        break
    if i % 50 == 0:
        print(f"res_norm = {res}")

plot_lcd_contour(vel0,xc,yc,interval=10)

In [ ]:
plot_lcd_compare(vel0,xc,yc,Re=1000,legend='calculated')